# BTC Price Analysis

This notebook demonstrates how to visualize the Bitcoin price, the Fear & Greed index, and the predicted price from our simple linear regression model.

In [ ]:
import sys
from pathlib import Path
import polars as pl
import matplotlib.pyplot as plt


In [ ]:
# Load processed data
data_path = Path("../data/processed/fear_and_greed_history_5min.parquet")
df = pl.read_parquet(data_path).drop_nulls()
df = df.sort("date")


In [ ]:
# Import training helper
src_path = Path("../python_train/src").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
from train import train_model
feature_cols = [
    "open_actual_value",
    "open_bitcoin_price_usd",
    "high_bitcoin_price_usd",
    "low_bitcoin_price_usd",
    "avg_actual_value",
]
model = train_model(df)
df = df.with_columns(pl.Series("predicted_price", model.predict(df.select(feature_cols).to_numpy())))


In [ ]:
# Plot actual vs predicted price and index
plt.figure(figsize=(14, 6))
plt.plot(df["date"].to_list(), df["close_bitcoin_price_usd"].to_list(), label="BTC Close")
plt.plot(df["date"].to_list(), df["predicted_price"].to_list(), label="Predicted", linestyle="--")
plt.plot(df["date"].to_list(), df["open_actual_value"].to_list(), label="Fear & Greed", alpha=0.5)
plt.legend()
plt.xlabel("Date")
plt.ylabel("Value")
plt.title("BTC Price vs Fear & Greed Index")
plt.tight_layout()
plt.show()


In [ ]:
# Compute correlations
import numpy as np
cor_fg = np.corrcoef(df["open_actual_value"].to_numpy(), df["close_bitcoin_price_usd"].to_numpy())[0,1]
cor_pred = np.corrcoef(df["predicted_price"].to_numpy(), df["close_bitcoin_price_usd"].to_numpy())[0,1]
print(f"Correlation (Fear & Greed vs BTC): {cor_fg:.3f}")
print(f"Correlation (Predicted vs BTC): {cor_pred:.3f}")
